In [1]:
import onnx
from onnxruntime.quantization import quantize_dynamic, quantize_static, QuantType, CalibrationDataReader,QuantFormat
import cv2
import numpy as np


In [2]:
model_path = '/container_root/ws/models/yolo11n-seg.onnx'
model = onnx.load('/container_root/ws/models/yolo11n-seg.onnx')

model_fp32 = model_path
model_quant = '/container_root/ws/models/yolo11n-seg.quant.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant)


In [3]:
!python -m onnxruntime.quantization.preprocess --input /container_root/ws/models/yolo11n-seg.onnx --output preprocessed.onnx

In [5]:
model_fp32 = 'preprocessed.onnx'
model_int8 = 'dynamic_quantized.onnx'

# Quantize 
quantize_dynamic(model_fp32, model_int8, weight_type=QuantType.QUInt8)

In [10]:
model_path = '/container_root/ws/models/yolo11n-seg.onnx'
model = onnx.load('/container_root/ws/models/yolo11n-seg.onnx')

# Class for Callibration Data reading
class ImageCalibrationDataReader(CalibrationDataReader):
    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.idx = 0
        self.input_name = "images"

    def preprocess(self, frame):
        # Same preprocessing that you do before feeding it to the model
        frame = cv2.imread(frame)
        X = cv2.resize(frame, (640, 640))
        image_data = np.array(X).astype(np.float32) / 255.0  # Normalize to [0, 1] range
        image_data = np.transpose(image_data, (2, 0, 1))  # (H, W, C) -> (C, H, W)
        image_data = np.expand_dims(image_data, axis=0)  # Add batch dimension
        return image_data

    def get_next(self):
        # method to iterate through the data set
        if self.idx >= len(self.image_paths):
            return None

        image_path = self.image_paths[self.idx]
        input_data = self.preprocess(image_path)
        self.idx += 1
        return {self.input_name: input_data}

# Assuming you have a list of image paths for calibration

# calibration_image_paths = ['./data/bus.jpg'] # you can add more of the image paths
mypath = '/container_root/ws/images_calibrating'
from os import listdir
from os.path import isfile, join
calibration_image_paths = [mypath + '/' + f for f in listdir(mypath) if isfile(join(mypath, f))]

print(calibration_image_paths)
# Create an instance of the ImageCalibrationDataReader
calibration_data_reader = ImageCalibrationDataReader(calibration_image_paths)

['/container_root/ws/images_calibrating/1341846655.242079.png', '/container_root/ws/images_calibrating/1341846650.550746.png', '/container_root/ws/images_calibrating/1341846650.650422.png', '/container_root/ws/images_calibrating/1341846654.706022.png', '/container_root/ws/images_calibrating/1341846648.570179.png', '/container_root/ws/images_calibrating/1341846654.670155.png', '/container_root/ws/images_calibrating/1341846653.530205.png', '/container_root/ws/images_calibrating/1341846648.470076.png', '/container_root/ws/images_calibrating/1341846655.106132.png', '/container_root/ws/images_calibrating/1341846652.290189.png', '/container_root/ws/images_calibrating/1341846652.090106.png', '/container_root/ws/images_calibrating/1341846650.246084.png', '/container_root/ws/images_calibrating/1341846651.890237.png', '/container_root/ws/images_calibrating/1341846648.138185.png', '/container_root/ws/images_calibrating/1341846648.805941.png', '/container_root/ws/images_calibrating/1341846652.4262

In [11]:
# Use the calibration_data_reader with quantize_static
quantize_static('preprocessed.onnx', "static_quantized.onnx",
				weight_type=QuantType.QInt8,
				activation_type=QuantType.QUInt8,
				calibration_data_reader=calibration_data_reader,
				quant_format=QuantFormat.QDQ,
				nodes_to_exclude=['/model.22/Concat_3', '/model.22/Split', '/model.22/Sigmoid'
													'/model.22/dfl/Reshape', '/model.22/dfl/Transpose', '/model.22/dfl/Softmax', 
													'/model.22/dfl/conv/Conv', '/model.22/dfl/Reshape_1', '/model.22/Slice_1',
													'/model.22/Slice', '/model.22/Add_1', '/model.22/Sub', '/model.22/Div_1',
													'/model.22/Concat_4', '/model.22/Mul_2', '/model.22/Concat_5'],
				per_channel=False,
				reduce_range=True,)